## Hand-written digit classification using nearest subspace  
2017-10-20 Jeff Fessler, University of Michigan  
2018-10-09 Julia 1.0.1 with Steven Whitaker  
2020-10-04 Julia 1.5.0 and added commentary - Caroline Crockett <br>
2021-10-01 Julia 1.6.0 and added commentary - Zongyu Li

## Group Formation and General Instructions
You may work individually, but we recommend that you work in pairs or groups of three. Find someone to work with and introduce yourself to them! One of you should copy the Google Document template and share the document with edit permissions with your group member(s): 
https://docs.google.com/document/d/1xHXktd0jroUftZ1jX9QJgxJjvyDTGnFJwLhfvZosDJc/template/preview (you must use your umich email address to access this google document). 

The Google Document will include your answers to a couple of questions that will be asked. For each question, pick someone to type up your **group's** response. After completing the response, the group member should sign their name by typing it below the answer. For the next question, have a different group member respond and sign their name in the same way. Keep rotating until all the questions have been answered.

The goal of this group exercise is to formulate your response as a group to the problem. When finished, **one** student in the group must submit a PDF of the google document to gradescope, entering the uniquenames of **all** students in the group. One group submits exactly one PDF, but we expect the PDF from different groups to differ. The deadline for submitting to gradescope will be announced on Canvas. Only submit the filled-in google document; do not submit a download of this Jupyter notebook. 

### Overview
This notebook illustrates how to classify handwritten digits
using a nearest-subspace approach.
We focus initially on just a pair of digits,
although later you will generalize to all digits. 

In [ ]:
# packages needed
using Plots
using LinearAlgebra # svd, norm, etc.
using Statistics: mean
using MIRTjim: jim
using MLDatasets 

### Part 0: load the data and look at it  
This time we are going to work with the 4 and 9 digits
because those are harder to separate.

In [ ]:
# read the MNIST data file for 4 and 9 digits
# download from web if needed
if !@isdefined(data)
    digitn = [4, 9]
    tmpx, tmpy = MNIST.traindata(Float32)
    data = n -> tmpx[:,:,findall(tmpy .== n)[1:1000]] # 1st 1000 of digit n
    data = 255 * cat(dims=4, data.(digitn)...)
    nx, ny, nrep, ndigit = size(data)
    data = data[:,2:ny,:,:] # make images non-square to force debug
    ny = size(data,2)
    @show size(data)
end
d0 = data[:,:,:,1]
d1 = data[:,:,:,2]

# Convert images to Float32 to avoid overflow errors
d0 = Array{Float32}(d0)
d1 = Array{Float32}(d1)

@show size(d0);

In [ ]:
# look at a couple of the images
jim(cat(d0[:,:,44:45], d1[:,:,654:655], dims=3))

In [ ]:
# use some data for training, and some for test
ntrain = 100
ntest = nrep - ntrain
train0 = d0[:,:,1:ntrain] # training data
train1 = d1[:,:,1:ntrain]
test0 = d0[:,:,(ntrain+1):end] # testing data
test1 = d1[:,:,(ntrain+1):end];

In [ ]:
# look at mean image from each class just to get a sense of things
mean0 = mean(train0, dims=3) # makes a 3D array 28x27x1
mean1 = mean(train1, dims=3)
@show size(mean0)
jim(cat(mean0, mean1, dims=3))

### Part 1: Scree plot
First, you will plot the singular values for both digits' training data on the same graph. 
This is called a "scree plot" - it can be helpful for choosing a dimension/rank.
We typically choose the rank at the "knee" in the plot - where the curve flattens outs. 

This is one exmample of why 551 is an engineering (not math) course. 
There is no single "correct" answer for where the knee in a scree plot is or how many singular vectors you should use to form your subspace. 
In practice, this is a judgement call and something that you can get better at with practice. 

For this part, you will turn in your scree plot and an explanation of where you think the knee is in your scree plot.

Hints: 
- you probably will need to call ```reshape``` here. Just like task 1, we are considering each image as a "vector" 

In [ ]:
# TODO: find the singular values and singular vectors for the training data 

# name your left singular vectors u0 and u1 and your singular values s0 and s1
# so that subsequent plot commands will work

# YOUR CODE HERE 
u0, s0, _ = ???
u1, s1, _ = ???

In [ ]:
# Make a scree plot: include this plot in your write-up 
# along with an explanation of where you think the knee is 
plot(1:ntrain, s0, line=(:dots, :blue), label="Digit 4")
plot!(1:ntrain, s1, line=(:dots, :red), label="Digit 9")

In [ ]:
#savefig("task3_part1_scree.png")

### Part 2: Look at the subspace basis vectors 

For this task, we will use 3 left singular vectors as the basis for our subspaces (for consistency and ease of grading). 
Next, you will display these first three left singular vectors of the training data for each digit. 
These are essentially very similar to the "principle components" in principle component analysis (PCA),
but keep in mind PCA is operated on "centered" data matrix, i.e., $(X-\mu)(X-\mu)'$, where $X$ is a $d \times n$ matrix,
$\mu = \frac{1}{n}\sum_{i=1}^n X_i$, $d$ is the dimension of features, $n$ is the number of data points. The subtraction $X-\mu$ is operated in column-wise.

Here in this task,
the idea behind the subspace classifier is
that to build a classifier that works on a lower dimensional subspace, i.e., defined by singular vectors.
In testing, for any given test vector, we first compute its projection onto
each subspace (# of subspaces = # of classes).
Then we choose the closest subspace (to that test vector)
and assign the corresponding label.
How to choose the closest subspace? 
Well, we need projection matrix (chapter 3).
For this part, you will turn in a plot of the first three left singular vectors for both digits and an explanation of whether the vectors look "sensible." 

In [ ]:
# TODO - complete this code
# We have given an example of how to show the images for the 4s
# You must also show the 9s 

r0, r1 = 3, 3 # make these variables so our code is more general! 
q0 = reshape(u0[:,1:r0], nx, ny, :)
q1 = ???

# show images of the 1st 3 components
p0 = jim(q0[:,:,1:3])
p1 = ??? 

# Combine both plots in a single output 
plot(p0, p1, layout=(2,1))

In [ ]:
#savefig("task3_part2_basis.png")

### Part 3: Explore how well the first left singular vectors separate the two classes  
Here we regress each of the training digits onto the first left singular vectors of the two classes, and then make a scatter plot to see how well separated they are.  
We will later use the first *3* left singular vectors to form our basis. We use only *1* in the code here just for easier visualization.

By "regress" we mean solve a linear LS problem of the form
$\hat{x} = \underset{x \in \mathbb{R}}{\arg \min} \lVert y - U x \rVert_2 = U'y$ (think about why), where: 
- $U \in \mathbb{R}^{27\cdot 28 \times 1}$ is the subspace basis (the first part of what you just visualized in the previous part)
- $y$ is a given training sample (think about its dimensions) 
- $x \in \mathbb{R}^1$ is a single linear coefficient 

You should see two clouds of points (these are projection coefficients, where the projection matrix is $UU'$) that are somewhat separated, but not perfectly so.
Include this plot in your report.
Also include an explanation of what the regress function returns. 
This plot illustrates that some high dimensional data are separable in lower dimension,
so we only need to build a classifier that works in lower dimension.

$\begin{align}
\hat{x} &= \underset{x \in \mathbb{R}}{\arg \min} \lVert y - U x \rVert_2
= \underset{x \in \mathbb{R}}{\arg \min} \lVert y - U x \rVert_2^2
\\
&= \underset{x \in \mathbb{R}}{\arg \min} (y'y - x'U'y - y'Ux + x'x) \quad \text{because } U'U=I
\\
&= \underset{x \in \mathbb{R}}{\arg \min} (y'UU'y - x'U'y - y'Ux + x'x) \quad \text{because }x \text{ is independent to }U \text{ and } y
\\
&= \underset{x \in \mathbb{R}}{\arg \min} \|x - U'y\|_2^2
=U'y.
\end{align}$

In [ ]:
# think about what the question mark should be
regress = (data, u) -> vec(mapslices(?, data, dims=(1,2)))
scatter(regress(train0, u0[:,1]), regress(train0, u1[:,1]), label="Digit 4")
scatter!(regress(train1, u0[:,1]), regress(train1, u1[:,1]), label="Digit 9")
plot!(xlabel = "Digit 4 " * "proj coeff of U[:,1]", ylabel = "Digit 9" * "proj coeff of U[:,1]", legend=:topleft)

In [ ]:
#savefig("task3_part3_scatter.png")

### Part 4: Classification based on nearest subspace  
This is the core of this project.  

You are going to take each test digit and project it onto the subspaces
defined by the first *three* left singular vectors
of each of the two digit's training data.
We will see in a later chapter why using a few of the left singular vectors is a reasonable choice of basis for a subspace in such applications.  

You will compute the distance of each test digit to those subspaces
and you will classify each digit by choosing the subspace that is closest.
Recall that if $Q_0$ is a matrix with orthonormal columns,
then the projection of a vector $y$ onto the subspace
Range($Q_0$) is simply
$\hat{y} = Q_0(Q_0'y)$,
and the distance between $y$ and that point is $|| \hat{y} - y ||_2$.  

Count how many of the test digits are classified correctly
and report the classification accuracy
for the 4's and the 9's in your report. 
Also comment on the classification accuracy in your report. 


Hints:
- The accuracies are roughly 80% and 90% (rounded to the nearest multiple of 10).
- Write what you are trying to solve in math before trying to code it
- ```reshape``` is again useful here.

In [ ]:
# TODO 
# Define variables y00, y01, y10, and y10 such that
# each variable is of size 28*27 x 900 (recall ntest=900)
# y01[:,i] corresponds to y-yhat for y=(the ith column of y0) when using Q1 as the basis
# y00[:,i] corresponds to y-yhat for y=(the ith column of y0) when using Q0 as the basis
# y11[:,i] corresponds to y-yhat for y=(the ith column of y1) when using Q1 as the basis
# y10[:,i] corresponds to y-yhat for y=(the ith column of y1) when using Q0 as the basis

# this can be done very efficiently (no for loops), 
# but you are welcome to use a for loop if you would like 
Q0 = reshape(q0, nx*ny, r0)
Q1 = reshape(q1, nx*ny, r1)

y0 = reshape(test0, nx*ny, :)
y00 = ?
y01 = ?

y1 = reshape(test1, nx*ny, :)
y10 = ?
y11 = ?

In [ ]:
# if you defined y00, y01, y10, and y11 correctly, 
# then this code will compute the classification accuracy! 
correct0 = (mapslices(norm, y00, dims=1) .< mapslices(norm, y01, dims=1))[:]
@show round(sum(correct0) / ntest, digits=4)

correct1 = (mapslices(norm, y10, dims=1) .> mapslices(norm, y11, dims=1))[:]
@show round(sum(correct1) / ntest, digits=4);

### Part 5: Find all the misclassified digits and display a couple of them  
  
Show pictures of the first two 4's and 9's that are misclassified in your report.

In [ ]:
bad0 = findall(correct0 .== false)
bad1 = findall(correct1 .== false)
@show length(bad0)
@show length(bad1)
p0 = jim(test0[?])
p1 = jim(test1[?])
plot(p0,p1, layout=(2,1))

In [ ]:
#savefig("task3_part5_misclassified.png")

### Optional Tasks   
* Go back to the previous classifiers (nearest angle and LS with hand-crafted features) for the "4" and "9" digits and compare the classification accuracies of those methods with this nearest subspace method.

* Try other subspace dimensions instead of just 3 to see if you can improve accuracy by using fewer or more left singular vectors.

* Generalize to nearest-subspace classification of all 10 digits.
This is fairly easy to do and we'll probably have the whole class do it
in a future discussion section.  The key to making it efficient is to use a 4D array of data of size [28,27,ntest,10] instead of separate variables data0, data1, data2, ... like we have used in this notebook.